## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Descubrimiento de grupos en los datos

---

En esta libreta se generan los modelos de clusterización sobre el dataset final del escenario para su posterior evaluación y análisis. Se utilizará como herramienta de soporte a mlflow para el registro completo de la experimentación. 

---

In [2]:
# Importación de librerías

import pandas as pd
# Se importa mlflow para registro de la experimentación 
import mlflow
# Se importan los métodos a utilizar para clusterizar
from sklearn.cluster import KMeans, AgglomerativeClustering, Birch
# Otras opciones: DBSCAN, MeanShift, BisectingKMeans

# Se importan las métricas a utilizar para evaluar el proceso
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from mlflow.models.signature import infer_signature

In [3]:
mlflow.set_tracking_uri("file:///C:/Users/monic/OneDrive - Universidad internacional de valencia/2º cuatri/13MBID- Metodologías de gestión y diseño de proyectos Big Data/Practica/13MBID-Actividades/mlruns")


----

##### Lectura del dataset

In [4]:
df = pd.read_csv("../../data/final/datos_finales_clusterizacion.csv", sep=";")
df.head(5)

,edad,importe_solicitado,duracion_credito,antiguedad_empleado,ingresos,pct_ingreso,tasa_interes,estado_credito,antiguedad_cliente,gastos_ult_12m,...,estado_cliente_ACTIVO,estado_cliente_PASIVO,genero_F,genero_M,nivel_educativo_DESCONOCIDO,nivel_educativo_POSGRADO_COMPLETO,nivel_educativo_POSGRADO_INCOMPLETO,nivel_educativo_SECUNDARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_INCOMPLETO
0,22,35000,3,123.0,59000,0.59,16.02,1,36.0,1088.0,...,True,False,False,True,False,False,False,False,True,False
1,21,1000,2,5.0,9600,0.10,11.14,0,39.0,1144.0,...,True,False,False,True,False,False,False,True,False,False
2,23,35000,2,4.0,65500,0.53,15.23,1,36.0,1887.0,...,True,False,False,True,False,False,False,False,True,False
3,24,35000,4,8.0,54400,0.55,14.27,1,54.0,1314.0,...,True,False,False,True,True,False,False,False,False,False
4,21,2500,2,2.0,9900,0.25,7.14,1,34.0,1171.0,...,True,False,True,False,False,False,False,True,False,False


In [5]:
# Transformación por las características de mlflow para la detección de nulos en campos de tipo integer
df = df.astype({col: 'float64' for col in df.select_dtypes(include='int').columns})

----

##### Configuración de la experimentación

In [14]:
# Se genera el experimento en mlflow
exp_name = 'Experimentación clustering #2 (SP7)'
exp_id = mlflow.create_experiment(name=exp_name)

In [15]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="Kmeans - K=2"):
    # Selección y configuración de la técnica
    modelo_clusters = KMeans(n_clusters=2)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 2)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    input_example = df.iloc[0].to_frame().T
    signature = infer_signature(df, cluster_labels)
    mlflow.sklearn.log_model(sk_model=trained_model, artifact_path="Kmeans_K2", input_example=input_example, signature=signature)

    # Se finaliza el registro
    mlflow.end_run()

2025/07/22 19:46:04 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Para contabilizar los elementos por cluster se utilizan las labels generadas

In [16]:
# Se preserva el conjunto de datos original para evitar problemas
df_kmeans_k2 = df.copy()
# Se agregan las labels generadas
df_kmeans_k2['cluster'] = cluster_labels
df_kmeans_k2['cluster'].value_counts()

cluster
1    8282
0     603
Name: count, dtype: int64

Se hace la exportación de los resultados a un archivo .csv nuevo

In [17]:
df_kmeans_k2.to_csv("../../data/final/df_kms_k2.csv", sep=";", index=False)

Se pasa a probar otro método (clustering jerárquico)

In [18]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="AGClustering - K=2"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=2)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 2)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    # input_example = df.iloc[0].to_frame().T
    signature = infer_signature(df, cluster_labels)
    mlflow.sklearn.log_model(trained_model, "AGC_2", signature=signature)

    # Se finaliza el registro
    mlflow.end_run()

2025/07/22 19:46:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/22 19:46:51 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


Para contabilizar los elementos por cluster se utilizan las labels generadas

In [19]:
# Se preserva el conjunto de datos original para evitar problemas
df_agc_k2 = df.copy()
# Se agregan las labels generadas
df_agc_k2['cluster'] = cluster_labels
df_agc_k2['cluster'].value_counts()

cluster
0    8514
1     371
Name: count, dtype: int64

Se realiza la exportación de los resultados

In [29]:
df_agc_k2.to_csv("../../data/final/df_agc_k2.csv", sep=";", index=False)

Prueba con otro método paramétrico

In [21]:
with mlflow.start_run(experiment_id=exp_id, run_name="Birch - K = 2"):
    modelo_clusters = Birch(n_clusters=2)
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Calcular métricas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 2)

    # Registrar métricas
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Guardar el modelo
    signature = infer_signature(df, cluster_labels)
    mlflow.sklearn.log_model(trained_model, "Birch_Model", signature=signature)

    mlflow.end_run()

2025/07/22 19:47:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


In [22]:
# Se preserva el conjunto de datos original para evitar problemas
df_bc = df.copy()
# Se agregan las labels generadas
df_bc['cluster'] = cluster_labels
df_bc['cluster'].value_counts()

cluster
0    8514
1     371
Name: count, dtype: int64

Se prueba con Kmeans pero con K = 3

Para contabilizar los elementos por cluster se utilizan las labels generadas

In [23]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="Kmeans - K=3"):
    # Selección y configuración de la técnica
    modelo_clusters = KMeans(n_clusters=3)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 3)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    input_example = df.iloc[0].to_frame().T
    signature = infer_signature(df, cluster_labels)
    mlflow.sklearn.log_model(trained_model, "Kmeans_K3", input_example=input_example, signature=signature)

    # Se finaliza el registro
    mlflow.end_run()

2025/07/22 19:48:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


In [24]:
# Se preserva el conjunto de datos original para evitar problemas
df_kms_k3 = df.copy()
# Se agregan las labels generadas
df_kms_k3['cluster'] = cluster_labels
df_kms_k3['cluster'].value_counts()

cluster
0    4691
1    3888
2     306
Name: count, dtype: int64

### Opciones de optimización del proceso

Utilizar PCA (solo para visualización de clusters) y un proceso de normalización de valores para eliminar el impacto de las diferencias en el dataset en la clusterización.

In [6]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Normalización de los datos
scaler = StandardScaler()
df_normalizado = scaler.fit_transform(df)

# Aplicación de PCA
pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_normalizado)

# Clusterización con las diferentes técnicas

In [16]:
# Se genera el experimento en mlflow
exp_name = 'Experimentación clustering #3'
exp_id = mlflow.create_experiment(name=exp_name)

In [17]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="Kmeans N - K=2"):
    # Selección y configuración de la técnica
    modelo_clusters = KMeans(n_clusters=2)
    
    trained_model = modelo_clusters.fit(df_normalizado)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df_normalizado, cluster_labels)
    score_db = davies_bouldin_score(df_normalizado, cluster_labels)
    score_ch = calinski_harabasz_score(df_normalizado, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 2)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    # input_example = df_normalizado.iloc[0].to_frame().T
    signature = infer_signature(df_normalizado, cluster_labels)
    mlflow.sklearn.log_model(trained_model, "Kmeans_K2_N", input_example=input_example, signature=signature)

    # Se finaliza el registro
    mlflow.end_run()

NameError: name 'input_example' is not defined

In [18]:
# Se preserva el conjunto de datos original para evitar problemas
# Convertir df_normalizado a un DataFrame de pandas
df_kms_k2_n = pd.DataFrame(df_normalizado, columns=df.columns)

# Se agregan las labels generadas
df_kms_k2_n['cluster'] = cluster_labels

# Contar los elementos por cluster
df_kms_k2_n['cluster'].value_counts()

cluster
0    6854
1    2031
Name: count, dtype: int64

Se utiliza el algoritmo jerárquico con datos normalizados

In [20]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="AGClustering - N- K=2"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=2)
    
    trained_model = modelo_clusters.fit(df_normalizado)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df_normalizado, cluster_labels)
    score_db = davies_bouldin_score(df_normalizado, cluster_labels)
    score_ch = calinski_harabasz_score(df_normalizado, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 2)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    # input_example = df.iloc[0].to_frame().T
    signature = infer_signature(df_normalizado, cluster_labels)
    mlflow.sklearn.log_model(trained_model, "AGC_2", signature=signature)

    # Se finaliza el registro
    mlflow.end_run()

2025/07/23 11:14:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/23 11:14:25 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [22]:
# Se preserva el conjunto de datos original para evitar problemas
df_normalizado = pd.DataFrame(df_normalizado, columns=df.columns, index=df.index)

df_agc_k2 = df_normalizado.copy()
# Se agregan las labels generadas
df_agc_k2['cluster'] = cluster_labels
df_agc_k2['cluster'].value_counts()

cluster
0    7209
1    1676
Name: count, dtype: int64

Se utiliza el algoritmo jerárquico con linkage = complete

In [23]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="AGClustering (complete)- K=2"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=2, linkage='complete')
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 2)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    # input_example = df.iloc[0].to_frame().T
    signature = infer_signature(df, cluster_labels)
    mlflow.sklearn.log_model(trained_model, "AGC_2", signature=signature)

    # Se finaliza el registro
    mlflow.end_run()

2025/07/23 11:28:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/23 11:28:17 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [24]:
# Se preserva el conjunto de datos original para evitar problemas
df_agc_k2 = df.copy()
# Se agregan las labels generadas
df_agc_k2['cluster'] = cluster_labels
df_agc_k2['cluster'].value_counts()

cluster
0    8884
1       1
Name: count, dtype: int64

Se utiliza el algoritmo jerárquico con linkage = average

In [25]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="AGClustering (average)- K=2"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=2, linkage='average')
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 2)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    # input_example = df.iloc[0].to_frame().T
    signature = infer_signature(df, cluster_labels)
    mlflow.sklearn.log_model(trained_model, "AGC_2", signature=signature)

    # Se finaliza el registro
    mlflow.end_run()

2025/07/23 11:33:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/23 11:33:19 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [26]:
# Se preserva el conjunto de datos original para evitar problemas
df_agc_k2 = df.copy()
# Se agregan las labels generadas
df_agc_k2['cluster'] = cluster_labels
df_agc_k2['cluster'].value_counts()

cluster
1    8868
0      17
Name: count, dtype: int64

Se utiliza el algoritmo jerárquico con linkage = single

In [27]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="AGClustering (single)- K=2"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=2, linkage='single')
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 2)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    # input_example = df.iloc[0].to_frame().T
    signature = infer_signature(df, cluster_labels)
    mlflow.sklearn.log_model(trained_model, "AGC_2", signature=signature)

    # Se finaliza el registro
    mlflow.end_run()

2025/07/23 11:34:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/23 11:34:49 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [28]:
# Se preserva el conjunto de datos original para evitar problemas
df_agc_k2 = df.copy()
# Se agregan las labels generadas
df_agc_k2['cluster'] = cluster_labels
df_agc_k2['cluster'].value_counts()

cluster
0    8884
1       1
Name: count, dtype: int64

## Prueba 3

In [30]:
# Se genera el experimento en mlflow
exp_name = 'Experimentación clustering #4'
exp_id = mlflow.create_experiment(name=exp_name)

Algoritmo Kmeans con K=2 y con inicialización de centroides más robusta

In [31]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="Kmeans - K=2 (centroides robustos)"):
    # Selección y configuración de la técnica con inicialización robusta
    modelo_clusters = KMeans(
        n_clusters=2,
        init='k-means++',  # Inicialización más robusta
        n_init=20,         # Ejecuta KMeans 20 veces con diferentes centroides y elige el mejor resultado
        random_state=42    # Para reproducibilidad
    )
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K y los ajustes nuevos
    mlflow.log_param('Valor K', 2)
    mlflow.log_param('init', 'k-means++')
    mlflow.log_param('n_init', 20)
    mlflow.log_param('random_state', 42)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    input_example = df.iloc[0].to_frame().T
    signature = infer_signature(df, cluster_labels)
    mlflow.sklearn.log_model(sk_model=trained_model, artifact_path="Kmeans_K2_robusto", input_example=input_example, signature=signature)

    # Se finaliza el registro
    mlflow.end_run()

2025/07/23 12:08:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


In [32]:
# Se preserva el conjunto de datos original para evitar problemas
df_kmeans_k2_robusto = df.copy()

# Se agregan las labels generadas por el modelo robusto
df_kmeans_k2_robusto['cluster'] = cluster_labels

# Visualización de la distribución de elementos por cluster
df_kmeans_k2_robusto['cluster'].value_counts()

cluster
0    8282
1     603
Name: count, dtype: int64

Algoritmo jerárquico con normalización MinMaxScatter

In [34]:
from sklearn.preprocessing import MinMaxScaler

# Escalado con MinMaxScaler
scaler = MinMaxScaler()
df_minmax = scaler.fit_transform(df)

In [35]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="AGClustering - K=2 - MinMaxScaler"):
    # Configuración del modelo
    modelo_clusters = AgglomerativeClustering(n_clusters=2)
    
    trained_model = modelo_clusters.fit(df_minmax)
    cluster_labels = trained_model.labels_

    # Cálculo de métricas
    score_s = silhouette_score(df_minmax, cluster_labels)
    score_db = davies_bouldin_score(df_minmax, cluster_labels)
    score_ch = calinski_harabasz_score(df_minmax, cluster_labels)

    # Registro de parámetros y métricas
    mlflow.log_param('Valor K', 2)
    mlflow.log_param('Scaler', 'MinMaxScaler')

    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Inferencia de firma para logging
    df_minmax_df = pd.DataFrame(df_minmax, columns=df.columns, index=df.index)
    signature = infer_signature(df_minmax_df, cluster_labels)
    mlflow.sklearn.log_model(trained_model, "AGC_2_MinMax", signature=signature)

    mlflow.end_run()

2025/07/23 12:19:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/23 12:19:27 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [36]:
# Se preserva el conjunto de datos original para evitar problemas
df_agc_k2 = df.copy()
# Se agregan las labels generadas
df_agc_k2['cluster'] = cluster_labels
df_agc_k2['cluster'].value_counts()

cluster
0    6857
1    2028
Name: count, dtype: int64